# Analysis of rules inferred by Bonita
#### Author: MG Palshikar
#### Date: 13 October, 2020
#### Goal: Unpack the *local_1.pickle* files created by Bonita into a human-readable format

***A set of output files ending with *local_1.pickle* are generated during the rule inference part of the Bonita pipeline. They contain model specifications and equivalent rules for each trial (ie, for each iteration of each network).
This analysis code collects data for each node, from each network/iteration and collates that data into a single dataframe. The dataframe includes information on the equivalent rules and network structure and is saved as a CSV file that can be opened in Excel or a text editor.***

## Load required packages and functions from the Bonita package

In [67]:
import networkx as nx
import cPickle as pickle
import scipy
import sys
import os
import pandas as pd
import re
import simulation
import requests
import numpy as np

"""import other pieces of Bonita software - utils.py, networkConstructor.py, analysis_accuracy.py"""
from utils import *
import networkConstructor as nc
from analysis_accuracy import *

## Function definitions

In [68]:
def getPathwayName (hsaURL):
    """Use KEGG API to get the readable name using the KEGG code (eg. hsa00010 corresponds to the glycolysis pathway)"""
    fileReg = re.compile('NAME\s+(\w+.*)')
    pathwayFile = requests.get('http://rest.kegg.jp/get/'+hsaURL, stream = True)
    for line in pathwayFile.iter_lines():
        result = fileReg.match(line)
        if result:
            return result.group(1)

In [69]:
def findEnds2(model, node, indiv):
    """ find the end of a node in the bitstring """
    node=model.nodeList.index(node)
    if node==len(model.nodeList)-1:
        end1=len(indiv)
    else:
        end1=model.individualParse[node+1]
    start1=model.individualParse[node]
    return start1, end1

In [70]:
def getNodeName(x, nodeList):
    return(nodeList[x])

In [71]:

def writeNode2(currentNode,nodeIndividual, model):
	"""write out evaluation instructions in BooleanNet format."""
	# This follows the exact same code as updateNode (for switch=0), but writes a string instead of actually updating the values of the nodes
	andNodes=model.andNodeList[currentNode] # find the list of shadow and nodes we must compute before computing value of current nodes
	andNodeInvertList=model.andNodeInvertList[currentNode] #find list of lists of whether input nodes need to be inverted (corresponds to inputOrder)
	writenode=''+model.nodeList[currentNode]+'*=' # set up the initial string to use to write node
	if model.andLenList[currentNode]==0 or sum(nodeIndividual)==0:
		return writenode + ' ' + model.nodeList[currentNode] #if no inputs, maintain value
	elif len(andNodes)==1: 
		#if only one input, then can either affect or not affect the node. so either keep the value or update to the single input's value
		value=''	
		#if only one input, then set to that number
		if andNodeInvertList[0][0]==0:
			value= value + model.nodeList[andNodes[0][0]]
		else:
			value= value+ 'not ' + model.nodeList[andNodes[0][0]]
		return writenode + value 
	else:
		#update nodes with more than one input

		# first deal with case of simple logic without need of linear regression
		orset=[]
		# go through list of possible shadow and nodes to see which ones actually contribute
		for andindex in range(len(nodeIndividual)):
			newval='('
			if nodeIndividual[andindex]==1:
				# if a shadow and contributes, compute its value using its upstream nodes
				if andNodeInvertList[andindex][0]:
					newval=newval+'not '
				newval=newval+model.nodeList[andNodes[andindex][0]]
				for addnode in range(1,len(andNodes[andindex])):
					newval= newval + ' and '
					if andNodeInvertList[andindex][addnode]:
						newval=newval+' not '
					newval=newval+model.nodeList[andNodes[andindex][addnode]]
				orset.append(newval +')')
			#combine the shadow and nodes with or operations
		writenode=writenode + orset.pop()
		for val in orset:
			writenode = writenode + ' or ' + val
		#print(writenode)
		return writenode


## Collect the *local1.pickle* files in the specified directory (change directory variable below)

In [72]:
directory = "" #os.getcwd() - use current working directory
outputfiles = []

for root, dirs, files in os.walk(directory): 
    for f in files:
        if f.endswith("_local1.pickle"): #if re.compile('.*hsa05418\_.+\local1.pickle').match(f):
            outputfiles.append(os.path.join(root,f))

print(outputfiles[0:4])

[]


## Open local1.pickle files and process the information into a single dataframe
**One row in the dataframe contains information for one node. The dataframe has the following columns:**
 - Network name - readable, descriptive KEGG network name
 - Method name - subfolder of the main directory in which the pickle was found
 - andNodeList - indices of parent nodes
 - andNodeInvertList - a bitstring encoding the activation and inhibition edges. True implies that the edge from the corresponding parent node in the andNodeList is an inhibitory edge
 - ruleLengths - length (ie, size) of the ERS for the node
 - equivs - bitstring representation of the equivalent rule set
 - plainRules - plain text representation of the rules in the ERS
 - randomERSIndividual - random individual from the ERS
 - minLocalSearchError - lowest error for the rules tried for each node

In [73]:
fileReg = re.compile('.*hsa(\d+)\_.+\.pickle')
seriesIndices = ["networkName", "methodName", "nodeList", "andNodeList", "andNodeInvertList", "ruleLengths", "equivs", "plainRules", "randomERSIndividual", "minLocalSearchError"]
df = pd.DataFrame(columns = seriesIndices)
i = 1

fileReg2 = re.compile(re.escape(directory) + r"(\w+.*)")
                      
for f in outputfiles[0:9]:
    getMethod=fileReg2.match(f)
    if getMethod:
        methodName=getMethod.group(1)
    else:
        methodName="N.A."
    result = fileReg.match(f)
    networkName = getPathwayName('hsa'+result.group(1))
    outputList = pickle.load(open(f, mode = "rb"))
    bruteOut1,dev,storeModel, storeModel3, equivalents, dev2 = [outputList[k] for k in range(len(outputList))]
    randomERSIndividual = bruteOut1 #random individual from the ERS
    minLocalSearchError = dev2 #lowest error for the rules tried for each node
    #equivalents = ERS for the network
    #storeModel = model from the GA
    minGAErrors = dev # minimum errors returned by the GA
    model1=modelHolder(storeModel3) #model from the local search
    for node in range(0,len(model1.nodeList)):
        plainRules=[]
        start1,end1=findEnds2(model1, model1.nodeList[node], equivalents[node])
        ers=equivalents[node] # find the bitstring for just this node
        inEdges=findInEdges(model1, model1.nodeList.index(model1.nodeList[node]))
        for rule in ers:
            plainRules.append(writeNode2(model1.nodeList.index(model1.nodeList[node]), rule, model1))
        ruleLengths=len(ers)
        ersAllNodes=plainRules
        s = pd.Series([networkName, methodName, model1.nodeList[node], model1.andNodeList[node], model1.andNodeInvertList[node], ruleLengths, str(ers), plainRules, randomERSIndividual, minLocalSearchError[node]], index = seriesIndices)
        df.loc[i] = s
        i = i + 1



## Save this dataframe to a CSV file with specified name (change 'CSVfile' variable below)

In [74]:
CSVfile = "local1Data.csv"
dfCSVFile = open(CSVfile, mode = "w")
df.to_csv(dfCSVFile)
dfCSVFile.close()

df.head()

,networkName,methodName,nodeList,andNodeList,andNodeInvertList,ruleLengths,equivs,plainRules,randomERSIndividual,minLocalSearchError
